In [1]:
from copy import deepcopy
import datetime
import re
import ssl
import urllib.request
import json

from lxml import etree


In [2]:
NS_MAP = {'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
          'aid': 'http://ns.adobe.com/AdobeInDesign/4.0/',
          'aid5': 'http://ns.adobe.com/AdobeInDesign/5.0/'}

# ignore the ssl certificate
context = ssl._create_unverified_context()

In [3]:
# get the start and end date of a sesion via the calendar API

def get_dates_from_session(session_code):
    """Return a tupple of start and end dates (as strings) of a session of parliament"""
    
    # Get the dates from API
    url = 'http://service.calendar.parliament.uk/calendar/sessions/list.json'
    response = urllib.request.urlopen(url, context=context)
    
    session_json = json.load(response)
    
    for session_obj in session_json:
        if session_obj['CommonsDescription'] == session_code:
            start_date = session_obj['StartDate']
            end_date = session_obj['EndDate']
            return(start_date[:10], end_date[:10])
    

In [4]:
# get the xml from the papers laid API and write to a file

# session code e.g. '2015-16'
# session_code = '2015-16'
session_code = '2016-17'

# dates of session of parliament
session_dates = get_dates_from_session(session_code)

url = ('http://services.paperslaid.parliament.uk/papers/list/daily.xml'
       f'?fromDate={session_dates[0]}&toDate={session_dates[1]}&house=commons')

response = urllib.request.urlopen(url, context=context)

as_downloaded_file_name = f'as_downloaded_papers_{session_code}.xml'
with open(as_downloaded_file_name, 'wb') as f:
    f.write(response.read())


In [5]:
# papers_xml_tree = etree.parse('daily.xml')
papers_xml_tree = etree.parse(as_downloaded_file_name)
papers_xml = papers_xml_tree.getroot()

In [6]:
# use a set to contain all the paper Ids. Sets are as in maths -- unique members
paper_ids = set()
papers_of_intrest = []

In [7]:
papers_xpath_result = papers_xml.xpath(
        # get all papers where the proceeding is not 'None'
        '/ArrayOfDailyPapers/DailyPapers/PublishedPapers/Paper'
        #  remove withdrawn papers there [sometimes DateWithdrawn does not agree with Withdrawn]
        '[Withdrawn[text()="false"]][DateWithdrawn[@xsi:nil="true"]]',
        namespaces=NS_MAP
    )

In [8]:
len(papers_xpath_result)

1797

In [9]:
# need to reverse the list of papers as we are more interested in the most recent papers
for paper in reversed(papers_xpath_result):

    # get the papers Id, ** I am assuming that these are unique to each paper **
    paper_id = paper.findtext('Id')
    
    if paper_id not in paper_ids:
        paper_ids.add(paper_id)
        papers_of_intrest.append(paper)
        


In [10]:
len(papers_of_intrest)

# use external script

1797